# MUTSIG analysis

Paths and prepare for finished cohorts

In [ ]:
import pandas as pd
import numpy as np
from scipy.io import loadmat
import matplotlib.pyplot as plt
# from adjustText import adjust_text
import glob as glob
import seaborn as sns


In [ ]:
# path variables
M1_MUTSIG_DIR = "/home/qingzhang/files/M1"
M2_MUTSIG_DIR =  "/home/qingzhang/files/M2"

In [ ]:
# read known driver genes
genes = pd.read_csv("/home/qingzhang/files/non_tier_4.tsv", sep = "\t")
genes.tier = genes.tier.astype('int64')
driver_genes = genes.gene[genes.tier.isin([1,2,3])].tolist()

# 1. Build master table for MAF and SIGGENES
prepare the subset of MAF reannotated from MutSig, prepare the siggenes from each cohort

In [ ]:
mut_eff_converter = {'1': 'ncd', 
                     '2': 'sil',
                     '3': 'mis',
                     '4': 'stp',
                     '5': 'spl' } 

def get_maf(cohort, mutect_version, gene_list=None, 
            cols = ['patient', 'symbol', 'chr', 'pos', 'newbase', 'effect', 'is_coding', 'context65']):
    
    # define base dir by version of mutect
    if mutect_version == 1:
        base_dir = M1_MUTSIG_DIR
        version = 'M1'
    elif mutect_version == 2:
        base_dir = M2_MUTSIG_DIR
        version = 'M2'
    
    
    # read files
    maf = pd.read_csv("{base_dir}/{cohort}/final_analysis_set.maf".format(base_dir = base_dir, cohort = cohort), sep = "\t")
    gene_dict = loadmat("{base_dir}/{cohort}/input_data.M/gene/name.mat".format(base_dir = base_dir, cohort = cohort))
    pat_dict = loadmat('{base_dir}/{cohort}/patients.mat'.format(base_dir = base_dir, cohort = cohort))
    
    # add patient name
    pat_name = []
    for ii in pat_dict['pat'][0][0][0][[i-1 for i in maf.pat_idx.tolist()]]:
        pat_name.append(ii[0][0])
    maf['patient'] = pat_name
    
    
    # add gene name
    gene_name = []
    for ii in gene_dict['tmp'][[i-1 for i in maf.gene_idx.tolist()]]:
        gene_name.append(ii[0][0])
    maf['symbol'] = gene_name
    
    # remap effects
    maf['effect_idx'] = maf['effect_idx'].astype('str')
    maf['effect'] = maf['effect_idx'].replace(mut_eff_converter)
    
    # remap newbase
    maf['newbase'] = maf["newbase_idx"].replace({1:"A", 2:"C", 3:"G", 4:"T"})
    # filter entries
    if gene_list is not None:
        maf = maf.loc[maf.symbol.isin(gene_list), cols ]
    else:
        maf = maf.loc[:, cols]
    
    maf['ttype'] = cohort
    maf['version'] = version
    
    raw_maf = pd.read_csv("{}/{}_full.maf".format(M2_RAWMAF_DIR, cohort), sep = '\t').rename({"newbase":"raw_nb"}, axis=1)
    raw_maf.patient = raw_maf.patient.str[-23:]
    maf = maf.merge(raw_maf.loc[:,["chr", "pos", "patient", "t_alt_count", "n_alt_count", "t_ref_count", "n_ref_count"]], how = 'left')
    

    return maf


In [ ]:
CODE_EFF = list(mut_eff_converter.values())

In [ ]:
def get_siggenes(cohort, mutect_version):
    if mutect_version == 1:
        res = loadmat('{}/{}/results.mat'.format(M1_MUTSIG_DIR, cohort), squeeze_me = True)
        version = "M1"
    elif mutect_version == 2:
        res = loadmat('{}/{}/results.mat'.format(M2_MUTSIG_DIR, cohort), squeeze_me = True)
        version = "M2"
#     print(res['G'])
    code_eff = list(mut_eff_converter.values())
    cols = ['gene', 'codelen', 'q'] + ['pCV', 'pCL', 'pFN'] + ["N" + eff for eff in code_eff] + ["n" + eff for eff in code_eff]
#     res = loadmat('{}/{}/results.mat'.format(M1_MUTSIG_DIR, cohort), squeeze_me = True)
    df = pd.DataFrame()
    for col in  cols:
        df[col] = res['G'][col].item()
    df['version'] = version
    df['ttype'] = cohort
    return df

In [ ]:
BIG_MAF = pd.DataFrame()
BIG_SIGGENE = pd.DataFrame()
# sig_gene_cols = ['gene', 'codelen','nncd', 'nmis', 'nstp', 'nspl', '',   'pCV', 'pCL', 'pFN', 'pCF', 'q']
for cc in finished_cohort:
    print(cc)
    BIG_MAF = pd.concat([BIG_MAF, pd.concat([get_maf(cc, 1), get_maf(cc, 2)], axis=0)], axis=0)
    BIG_SIGGENE = pd.concat([BIG_SIGGENE, pd.concat([get_siggenes(cc, 1), 
                                                     get_siggenes(cc, 2)], axis=0)], axis=0)


In [ ]:
BIG_MAF = BIG_MAF.reset_index()
BIG_SIGGENE = BIG_SIGGENE.reset_index()

In [ ]:
FILTERS = pd.DataFrame()
for cc in finished_cohort:
    filter_reasons = pd.read_csv("/demo-mount/m2_results/combined_res/filters/{}.tsv".format(cc), sep = '\t').loc[:, ['chr', 'pos', 'patient', 'filters']]
    filter_reasons.patient = filter_reasons.patient.str[-23:]
    filter_reasons.chr = filter_reasons.chr.astype('int64')
    filter_reasons.columns = ["chr", "pos", "patient", "filters"]
    filter_reasons['ttype'] = cc
    FILTERS = pd.concat([FILTERS, filter_reasons], axis=0)
FILTERS
BIG_MAF = BIG_MAF.merge(FILTERS, how = "left")

Alternatively, read directly from the pickle file.

In [ ]:
import pandas as pd
import numpy as np
BIG_MAF = pd.read_pickle("/home/qingzhang/files/BIG_MAF.pkl")
BIG_SIGGENE = pd.read_pickle("/home/qingzhang/files/BIG_SIGGENE.pkl")

In [ ]:
PON = pd.read_csv("/home/qingzhang/files/Mutect2-exome-panel.vcf", sep = "\t", 
                  comment = "#", header = None, usecols = [0,1]).rename({0:"chr", 1:"pos"}, axis=1)
PON = PON.loc[PON.chr != "MT", :]
PON.chr = PON.chr.replace({"X":23, "Y":24}).astype("int64")

# Merged sets of variants and genes

## MERGED MAF

In [ ]:
M1_callset = BIG_MAF.loc[(BIG_MAF.version == "M1")]
M2_callset = BIG_MAF.loc[(BIG_MAF.version == "M2")]# & (BIG_MAF.t_alt_count > 3)]
MERGED_calls = M1_callset.merge(M2_callset, how = 'outer', 
                       on = ['ttype','symbol', 'chr','pos','patient', 'filters'], 
                       suffixes  =['_M1', "_M2"])
# MERGED_calls = MERGED_calls.merge(FILTERS, how = 'left', on = ['ttype', 'patient', 'chr', 'pos'])
MERGED_calls["called_by"] = "both"
MERGED_calls["called_by"][(~MERGED_calls.effect_M1.isnull()) & MERGED_calls.effect_M2.isnull() ] = "M1_only"
MERGED_calls["called_by"] [(~MERGED_calls.effect_M2.isnull()) & MERGED_calls.effect_M1.isnull() ] = "M2_only"

* Number of M1-only sites in PoN

In [ ]:
PON = pd.read_csv("/home/qingzhang/files/Mutect2-exome-panel.vcf", sep = "\t", 
                  comment = "#", header = None, usecols = [0,1]).rename({0:"chr", 1:"pos"}, axis=1)
PON = PON.loc[PON.chr != "MT", :]
PON.chr = PON.chr.replace({"X":23, "Y":24}).astype("int64")
PON['inpon'] = "m2_pon"
MERGED_calls = MERGED_calls.merge(PON, how = "left")

In [ ]:
MERGED_calls.inpon.value_counts()

In [ ]:
MERGED_calls.called_by.value_counts()

In [ ]:
1966/87907

* Venn plot

In [ ]:
def venn(data):
    total = len(data['effect_M1'])
    d = {}
#     d['M1_only'] = np.sum( (data['effect_M2'].isnull()) & (~data['effect_M1'].isnull())) / total
    d['M2_only'] = np.sum( (data['effect_M1'].isnull()) & (~data['effect_M2'].isnull())) / total
    d['called_both'] = np.sum( (~data['effect_M2'].isnull()) & (~data['effect_M1'].isnull())) / total
#     d['M1_w_filter'] = np.sum((~data['effect_M1'].isnull()) & (data['effect_M2'].isnull()) & (~data['filters'].isnull())  ) / total
#     d['M1_wo_filter'] = np.sum((~data['effect_M1'].isnull()) & (data['effect_M2'].isnull()) & (data['filters'].isnull())  ) / total
#     d['total_patient'] = data['patient'].nunique()
    return pd.Series(d)

# sns.set(style="whitegrid")
df = BIG_MAF.loc[BIG_MAF.version == "M2"].groupby(['ttype', 'patient']).size().reset_index().rename({0:"counts"}, axis=1)
plot_order = df.groupby("ttype").counts.mean().sort_values().index.tolist()
df['counts'] = np.log10(df['counts'])

fig = plt.figure(figsize = (10,10))
ax = fig.add_axes([0.1, 0.5, 0.8, 0.4])
ax2 = fig.add_axes([0.1, 0.2, 0.8, 0.2])
df_venn = MERGED_calls.groupby('ttype').apply(venn).loc[plot_order,:]
df_venn.plot.bar(stacked = True, ax=ax, cmap = "Blues", linewidth=1, edgecolor = 'grey')
ax.set_xticklabels(plot_order, rotation =45)
leg = ax.legend(loc='lower right', bbox_to_anchor=(0.7, 0, 0.5, 1))
leg.get_frame().set_linewidth(0.0)
ax.set_title("Call set Venn", fontsize='large')
ax.set_xlabel("")
ax.set_ylabel('fraction')

for i, (tt, num) in enumerate(zip(plot_order, df_venn.called_both)):
    num = "{:.0%}".format(num)
    ax.text(i-0.25, 0.5, num, fontsize=8)
# fig.tight_layout()
ax2.plot(BIG_MAF.groupby('ttype').patient.nunique()[df_venn.index], color = "tab:grey", linestyle = "-")
ax2.set_xticklabels(plot_order, rotation =45)
ax2.set_title("Cohort attribute (ordered by average #variant called per patient by M2)", fontsize='large')
ax2.set_ylabel('# patient', color="tab:grey")
ax3 = ax2.twinx()  # instantiate a second axes that shares the same x-axis

color = 'tab:blue'


# ax3.set_yscale('log')
sns.violinplot('ttype', 'counts', ax = ax3, color = color, data = df,
              order=plot_order)

ax3.tick_params(axis='y', labelcolor=color)

ax3.set_xticklabels(ax3.get_xticklabels(), rotation =45)
ax3.set_ylabel('lg(# calls per patient)', color=color)  # we already handled the x-label with ax1
sns.despine()
plt.savefig("venn", dpi=300, bbox_inches='tight')



In [ ]:
df_venn = df_venn.reset_index()
df_venn["filter_ratio"] = df_venn["M1_w_filter"] / (df_venn["M1_w_filter"] + df_venn["M1_wo_filter"])

* minimum alt support

In [ ]:
def set_cutoff(cut):
    M1_callset = BIG_MAF.loc[BIG_MAF.version == "M1"]
    M2_callset = BIG_MAF.loc[(BIG_MAF.version == "M2") & (BIG_MAF.t_alt_count > cut)]
    MERGED_calls = M1_callset.merge(M2_callset, how = 'outer', 
                           on = ['ttype','symbol', 'chr','pos','patient'], 
                           suffixes  =['_M1', "_M2"])
#     MERGED_calls = MERGED_calls.merge(FILTERS, how = 'left', on = ['ttype', 'patient', 'chr', 'pos'])
    df_venn = MERGED_calls.groupby('ttype').apply(venn).reset_index().sort_values(["called_both", "M2_only"])
    df_venn['cutoff'] = cut
    return df_venn

df_test = pd.DataFrame()
for cc in range(1, 7):
    df_test = pd.concat([df_test, set_cutoff(cc)], axis=0)

In [ ]:
df_test.groupby("cutoff").called_both.mean()

In [ ]:
df_test[df_test.cutoff == 5]

In [ ]:
plt.figure(figsize=(8, 6))
df_test['cutoff'] = df_test['cutoff'] + 1
ax = sns.lineplot(x = "cutoff", y = "called_both", hue = "ttype", data=df_test, legend= "brief", marker="o")
# plt.legend(loc='upper left', , bbox_to_anchor=(0.75, 0, 0.5, 1))
# plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1.05, 1), loc=2)
plt.title("Filter M2 calls with different alt count cutoff")

sns.despine()
ax.set_ylabel('fraction of variants called by both caller')
ax.set_xlabel('minimum alt count required')
plt.savefig("test_cutoff", dpi=150, bbox_inches='tight')

* Figure: filters

In [ ]:
M1_with_filter = MERGED_calls.loc[~MERGED_calls.filters.isnull(), ["patient", "symbol", "chr", "pos", "filters", "ttype", "inpon"]]
M1_with_filter["filters"] = M1_with_filter.filters.str.split(";" )
M1_with_filter = M1_with_filter.explode("filters")

In [ ]:
M1_with_filter.filters.value_counts()

In [ ]:
mat = M1_with_filter.groupby(['ttype', 'filters']).size().reset_index().pivot("ttype", "filters").fillna(0)
mat.columns = mat.columns.droplevel(0)
mat = mat.apply(lambda row: row / np.sum(row), axis=1)
g = sns.clustermap(mat, cmap = "Blues", figsize=(10, 8))
col_ids = g.dendrogram_col.reordered_ind
row_ids = g.dendrogram_row.reordered_ind
f, ax = plt.subplots(figsize=(12, 9))
g = sns.heatmap(mat.iloc[row_ids, col_ids], cmap = "Blues", ax=ax, linewidths=0.4)

# Rotate the tick labels and set their alignment.
plt.setp(g.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
g.set_xticklabels(g.get_xticklabels(), rotation=45)
g.set_yticklabels(g.get_yticklabels(), rotation=45)
b, t = plt.ylim() # discover the values for bottom and top
b += 0.5 # Add 0.5 to the bottom
t -= 0.5 # Subtract 0.5 from the top
plt.ylim(b, t) # update the ylim(bottom, top) values
plt.savefig("clustering_filter_new", dpi=300, bbox_inches='tight')

* Lego (dependent on Capy)

## MERGED_SIG

In [ ]:
M1_SIG = BIG_SIGGENE.loc[BIG_SIGGENE.version == "M1"].drop(['version'], axis=1)
M2_SIG = BIG_SIGGENE.loc[BIG_SIGGENE.version == "M2"].drop(['version'], axis=1)
MERGE_SIG = M1_SIG.merge(M2_SIG, how = 'outer', 
                       on = ['ttype','gene', 'codelen'], 
                       suffixes  =['_M1', "_M2"])

* The four major stata for downstream analysis

In [ ]:
def categorize_genes(ttype):
    sub_df = MERGE_SIG.loc[MERGE_SIG.ttype == ttype]
#     print(sub_df.head())
    d = {}
    d['M1_KCG'] = sub_df.gene[(sub_df.q_M1 < 0.1) & (sub_df.q_M2 > 0.1) & (sub_df.gene.isin(driver_genes))].tolist()
    d['M2_KCG'] = sub_df.gene[(sub_df.q_M1 > 0.1) & (sub_df.q_M2 < 0.1) & (sub_df.gene.isin(driver_genes))].tolist()
    d['M1_nKCG'] = sub_df.gene[(sub_df.q_M1 < 0.1) & (sub_df.q_M2 > 0.1) & ~(sub_df.gene.isin(driver_genes))].tolist()
    d['M2_nKCG'] = sub_df.gene[(sub_df.q_M1 > 0.1) & (sub_df.q_M2 < 0.1) & ~(sub_df.gene.isin(driver_genes))].tolist()
    return d

cats = {}
for cc in finished_cohort:
    cats[cc] = categorize_genes(cc)
# cats

In [ ]:
def summarize_muts():
    df = MERGE_SIG[(MERGE_SIG.q_M2 < 0.1) & (MERGE_SIG.q_M1 > 0.1)]
    for eff in CODE_EFF:
        df['d{}'.format(eff)] = df['n{}_M2'.format(eff)]-df['n{}_M1'.format(eff)]
    return df.loc[:,['gene', 'ttype'] + ['d{}'.format(eff) for eff in CODE_EFF]]
                   

sig_diff = pd.melt(summarize_muts(), 
                   id_vars=["gene", "ttype"], value_vars=["dncd", "dsil", "dmis", "dstp", "dspl"])

In [ ]:
sig_diff

* The scale of different types of calls per gene for differential significant genes

In [ ]:
plt.figure(figsize = (8, 16))
sns.catplot(data = sig_diff, x = "ttype", y = "value", hue = "variable",row = "variable",
            alpha = 0.5, sharey = False, 
           height = 2, aspect=3)

# 2. Seaborn grid plots comparing variants

- variant counts stratified by patients
- variant counts stratified by patients & variant type
- log ratio of coding/noncoding mutations

In [ ]:
# Utils
## add a y=x reference line
def add_y_equals_x( **kwargs):
    ax = plt.gca()
    lims = [
        np.min([ax.get_xlim(), ax.get_ylim()]),  # min of both axes
        np.max([ax.get_xlim(), ax.get_ylim()]),  # max of both axes
    ]
    ax.plot(lims, lims, 'k-', alpha=0.2, zorder=0)
    ax.set_aspect('equal')
    ax.set_xlim(lims)
    ax.set_ylim(lims)

In [ ]:
def single_n(var, cohorts = None):
    df_p = BIG_MAF.groupby(['ttype', var,  'version'])['effect'].count().unstack('version').reset_index()    
    
    if var == "symbol":
        df_p['mark'] = '1'
        df_p['mark'][(df_p.ttype == "DLBC") & (df_p.symbol == "PIM1")] = 2
        df_p['mark'][(df_p.ttype == "THYM") & (df_p.symbol == "GTF2I")] = 2
        g = sns.FacetGrid(df_p, col="ttype", col_wrap=4, sharex=False, sharey=False, hue = "mark", aspect=1)
    else:
        g = sns.FacetGrid(df_p, col="ttype", col_wrap=4, sharex=False, sharey=False, aspect=1)
        
    if cohorts is None:
        pass
    else:
        df_p = df_p.loc[df_pe.ttype.isin(cohorts),:]
    
    
    
    g.map(plt.scatter, "M1", "M2", alpha = 0.5)
    g.map(add_y_equals_x)
    plt.subplots_adjust(top=0.95)
    g.set_titles("{col_name}")
    
    if var == "symbol":
        var = "gene"
    g.fig.suptitle('Variant counts per {}'.format(var))
    plt.xlabel("M1")
    plt.ylabel("M2")
    plt.savefig("scatter_by_{}".format(var), dpi=150)
    return df_p

In [ ]:
df_g = single_n("symbol")

In [ ]:
import seaborn as sns
df_p = single_n("patient")

In [ ]:
df_pe = BIG_MAF.groupby(['ttype', 'symbol', 'version'])['effect'].value_counts().unstack('version').reset_index().fillna(0)

In [ ]:
# BIG_MAF['neighboring_indel'] = "no"
# BIG_MAF['neighboring_indel'][BIG_MAF['dist_indel'] < 25] = "yes"
def gene_type_n(cohorts = None):
    df_pe = BIG_MAF.groupby(['ttype', 'symbol', 'version'])['effect'].value_counts().unstack('version').reset_index().fillna(0)
    if cohorts is None:
        pass
    else:
        df_pe = df_pe.loc[df_pe.ttype.isin(cohorts),:]
    g = sns.FacetGrid(df_pe, col="ttype", col_wrap=4, 
                      sharex=False, sharey=False, aspect=1, hue = "effect",
                      hue_kws=dict(marker=["^", "v", "*", "o", "+"]),
                      hue_order =CODE_EFF)
    g = (g.map(plt.scatter, "M1", "M2", alpha = 0.3)).set(yscale = 'log', xscale = 'log') 
    g.map(add_y_equals_x).add_legend()
    g.set_titles("{col_name}", fontsize="large")
#     plt.legend(fontsize='x-large', title_fontsize='40')
    plt.setp(g._legend.get_texts(), fontsize='15') # for legend text
    plt.setp(g._legend.get_title(), fontsize='15') # for legend title
#     g.fig.suptitle('Variants counts per gene, each variant type', fontsize = "large")
    plt.savefig("patient_gene", dpi=150, bbox_inches='tight')
    plt.subplots_adjust(top=0.95)

In [ ]:
gene_type_n()

In [ ]:
def plot_ratio():
    BIG_MAF["is_coding2"] = BIG_MAF.effect != CODE_EFF[0] #denotes flanking

    df_ratio = BIG_MAF.groupby(['ttype', 'patient', 'version', 'is_coding2']).agg('size').unstack('is_coding2').reset_index()
    df_ratio.columns = ['ttype', 'patient', 'version', 'noncoding', 'coding']
    df_ratio['ratio'] = np.log(df_ratio['coding']) - np.log(df_ratio['noncoding'])
    df_ratio = df_ratio.pivot_table(index = ['ttype', 'patient'], columns = 'version', values = 'ratio').reset_index()
    g = sns.FacetGrid(df_ratio, col="ttype", col_wrap=4, 
                  sharex=False, sharey=False, aspect=1)
    g.map(sns.distplot,"M1", hist = False, color = 'blue', label = 'M1')
    g.map(sns.distplot,"M2", hist = False, color = 'red', label = "M2")
    # g.map(add_y_equals_x)
    g.set_titles("{col_name}")
    g.add_legend()
    g.fig.suptitle('log(coding)-log(noncoding)')
    plt.subplots_adjust(top = 0.9, hspace = 0.5)
    return df_ratio
    

In [ ]:
ncd_ratio = plot_ratio()

# prioritize genes

In [ ]:
MERGED_calls.head()

In [ ]:
MERGED_calls.dtypes

In [ ]:
df = MERGED_calls[~MERGED_calls.effect_M2.isnull()]

In [ ]:

df['tumor_VAF'] = df['t_alt_count_M2'] / (df['t_alt_count_M2']+ df['t_ref_count_M2'])

In [ ]:
sns.set(font_scale=1)
sns.set_style("ticks")
MERGED_calls['tumor_VAF'] = MERGED_calls["t_alt_count_M2"] / (MERGED_calls["t_alt_count_M2"] + MERGED_calls["t_ref_count_M2"])
g = sns.FacetGrid(MERGED_calls[MERGED_calls.called_by.isin(["both", "M2_only"])], col="ttype",  hue="called_by", col_wrap=4, 
                  sharex = True, sharey = False)
g.map(plt.scatter, "tumor_VAF", "t_alt_count_M2", alpha=0.2)
g.set(yscale = 'log').set(ylim=(0, 20))#.add_legend()
g.set_titles("{col_name}")
g.map(plt.axhline, y=2, xmin=0, xmax=1, color = "red")
plt.subplots_adjust(top=0.95)
sns.despine()
g.fig.suptitle('VAF vs. tumor alt coverage')
g.set_titles("{col_name}")
plt.savefig("tumor_AF_facet", dpi=300)

In [ ]:
MERGE_SIG['gene_type'] = "none"
MERGE_SIG['gene_type'][(MERGE_SIG.q_M1 < 0.1) & (MERGE_SIG.q_M2 > 0.1)] = "M1_only"
MERGE_SIG['gene_type'][(MERGE_SIG.q_M1 > 0.1) & (MERGE_SIG.q_M2 < 0.1)] = "M2_only"
MERGE_SIG['gene_type'][(MERGE_SIG.q_M1 < 0.1) & (MERGE_SIG.q_M2 < 0.1)] = "both"

In [ ]:
sig = MERGE_SIG[MERGE_SIG.ttype == "BRCA"]
sig = sig.loc[:,['gene', "pCL_M1", "pCL_M2", 'gene_type']].dropna()
sig.max()

In [ ]:
MERGE_SIG.head()

## MutSig plots

In [ ]:
BIG_SIGGENE

In [ ]:
# MutSig
pvals = BIG_SIGGENE.pivot_table(index = ['ttype', 'gene'], columns='version', values='q').reset_index()
pvals['gene_type'] = "consistent"
pvals['gene_type'][(pvals.M1 <= 0.1) & (pvals.M2 >= 0.1)] = "M1-only"
pvals['gene_type'][(pvals.M1 >= 0.1) & (pvals.M2 <= 0.1)] = "M2-only"
pvals.head()

In [ ]:
pvals_long = pd.melt(pvals[(pvals.M1 != 1) | (pvals.M2 != 1)], id_vars=['ttype', 'gene', 'gene_type'], value_vars=['M1', 'M2'])
list_ordering = ["M1-only", 'M2-only', "consistent"]
pvals_long["gene_type"] = pd.Categorical(pvals_long["gene_type"], categories=list_ordering) 
pvals_long = pvals_long.sort_values("gene_type").reset_index()

In [ ]:
from scipy import stats


def scatter(x, y, g,  **kwargs):
    sns.lineplot(x, y, alpha = 0.5, units = g, markers = True, estimator=None, lw=1, legend = "brief", 
                 palette=sns.color_palette("coolwarm", 3),
                 **kwargs).set(yscale='log', ylabel="q value")
g = sns.FacetGrid(pvals_long , col="ttype",  hue = "gene_type", col_wrap=6, aspect=0.5,
                  col_order=["CHOL", "PCPG", "UCS", "KICH", 'UVM', 'KICH', 'ACC', 'SARC', 'MESO', 'THYM', 'ESCA', 
                             'BLCA', 'BRCA', 'CESC', 'UCEC', 'DLBC', 'SKCM', 'LIHC'],
                 palette=sns.xkcd_palette(["blue", "grey", "red"]), hue_order = ['M2-only', "consistent", "M1-only"])
g.map(scatter, "version", "value", "gene")
g.fig.suptitle('MutSig Q value from two call sets')
g.add_legend()
g.set_titles("{col_name}").set(xlabel=None, ylabel="Q value")
plt.subplots_adjust(top = 0.9, hspace = 0.5)
sns.despine(top = True, right = False)
plt.savefig("qval_combined", dpi=300)

In [ ]:
pvals

In [ ]:
def plot_coverage(cohort, cap = 20, log = False):
    raw_maf = pd.read_csv("{}/{}_full.maf".format(M2_RAWMAF_DIR, cohort), sep = '\t')
    select_maf = M2_FINAL_SET.loc[M2_FINAL_SET.ttype == cohort, :].merge(raw_maf, how = "left" )
    
    
    
    raw_maf['patient'] = raw_maf['patient'].str[-20:]
    fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (12, 6), sharey = True)
    fig.suptitle("{}: coverage".format(cohort))
    def plot_two(cohort, v):
        final_maf = get_maf(cohort = cohort, mutect_version=v)
        final_maf['patient'] = final_maf['patient'].str[-20:]
        df = final_maf.merge(raw_maf, how = 'left', on = ['chr', 'pos', 'patient']).dropna().loc[:,['patient', 'chr', 'pos', 'symbol', 'is_coding', 't_alt_count', 't_ref_count']]
        df['coverage'] = df['t_alt_count'] + df['t_ref_count']
        df['ratio'] = df['t_alt_count'] / df['coverage']
        
        axes[v-1].scatter(df.loc[df.is_coding == 1, ]['ratio'], 
                          df.loc[df.is_coding == 1, ]['t_alt_count'], 
                          alpha = 0.1,
                          label = 'M{} coding'.format(v))
        axes[v-1].scatter(df.loc[df.is_coding == 0, ]['ratio'], 
                          df.loc[df.is_coding == 0, ]['t_alt_count'], 
                          alpha = 0.1,
                          label = 'M{} non-coding'.format(v))
        axes[v-1].legend(loc = 'lower right')
        axes[v-1].set_yscale('log')
        axes[v-1].set_title("M{}".format(v))
        axes[v-1].set(ylim = [0.5, cap], xlabel = "alt/total ratio", ylabel = "coverage")
        axes[v-1].set_yticks(np.array([1, 2, 5, 10, 15, 20]))
        axes[v-1].set_yticklabels(np.array([1, 2, 5, 10, 15, 20]))
        
        return None
    
    plot_two(cohort, 1)
    plot_two(cohort, 2)
    if log:
        cap = str(cap) + '-log'
    plt.savefig('/demo-mount/m2_results/combined_res/plots/coverage/{}-{}.png'.format(cohort, cap), dpi = 80)

plot_coverage("BRCA", log = True)

In [ ]:
# from seaborn_qqplot import qqplot
import matplotlib.patches as mpatches

from scipy.stats import uniform
import scipy.stats as stats
runif = np.random.uniform(20000)
colors = {'M1':'red', 'M2':'blue', 
          'both':'orange', 'none':'grey'}
big_size = 25
small_size = 2
sizes = {'M1':big_size, 'M2':big_size, 
          'both':big_size, 'none':small_size}
def qqplot(x, y, c, s, **kwargs):
#     x = np.random.uniform(0, 1, len(y))
    _, xr = stats.probplot(x, dist=stats.uniform, fit=False)
    _, yr = stats.probplot(y, dist=stats.uniform, fit=False)
    plt.scatter(-np.log(xr), -np.log(yr), c=c, s = s, alpha = 0.7)
    
    


# ax.scatter(df['carat'], df['price'], c=df['color'].apply(lambda x: colors[x]))
def compare_pvalues(cohort):
    
    # sig genes set
    BIG_SIGGENE['q'] = BIG_SIGGENE.q.fillna(1)
    M1_sig_genes = BIG_SIGGENE.gene[(BIG_SIGGENE.q < 0.1) & (BIG_SIGGENE.version == "M1") & (BIG_SIGGENE.ttype == cohort)].tolist()
    M2_sig_genes = BIG_SIGGENE.gene[(BIG_SIGGENE.q < 0.1) & (BIG_SIGGENE.version == "M2") & (BIG_SIGGENE.ttype == cohort)].tolist()

    df = pd.melt(BIG_SIGGENE.loc[(BIG_SIGGENE.ttype == cohort)], 
                                 value_vars=['pCV', 'pCL', 'pFN'], 
                                 id_vars=['gene', 'version']).fillna(1)
    df['caller']  = df['version']
    df['sig'] = 'none'
    df['sig'][df['gene'].isin(list(set(M1_sig_genes) - set(M2_sig_genes)))] = 'M1'
    df['sig'][df['gene'].isin(list(set(M2_sig_genes) - set(M1_sig_genes)))] = 'M2'
    df['sig'][df['gene'].isin(list(set(M1_sig_genes) & set(M2_sig_genes)))] = 'both'
    df['cc'] = [colors[x] for x in df['sig'].tolist()]
    df['size'] = [sizes[x] for x in df['sig'].tolist()]
#     df['alpha'] = [alphas[x] for x in df['sig'].tolist()]
    df['unif'] = np.random.uniform(0, 1, df.shape[0])
    g = sns.FacetGrid(df, col="variable", col_order=['pFN','pCL', 'pCV'],
                      row = 'caller', row_order = ['M1', 'M2'],
#                       hue = 'sig', #hue_kws={"marker": ["s", "D"]},
#                       hue = 'sig', hue_order = ['none', 'both', 'M1-only', 'M2-only'],
                      height=4, sharey=False, sharex=False)
    g.map(qqplot,'unif', "value", "cc", "size")

#   g.add_legend()
    g.map(add_y_equals_x)
    g.fig.text(0.5, 0.05, "-log10(Expected p-value)", ha="center", fontsize=12)
    g.fig.text(0.07, 0.35, "-log10(Observed p-value)", rotation=90, ha="center", fontsize=12)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)
    g.fig.suptitle(cohort)
    patch_list = [mpatches.Patch(color=colors[sig_type], label=sig_type) for sig_type in colors.keys()]
    g.add_legend(handles=patch_list, title = "Significant by:", loc=7)
    plt.savefig("/home/qingzhang/files/combined_pvals-{}.png".format(cohort), dpi=150)
    return(df)



In [ ]:
compare_pvalues("SKCM")

In [ ]:
for cc in finished_cohort:
    compare_pvalues(cc)

In [ ]:

categories = ["M1_KCG", "M1_nKCG", "M2_KCG", "M2_nKCG"]
def gene_category(df, cats):
    df['cate_ids'] = "others"
    for tumor in cats.keys():
        for cate in categories:
            df.cate_ids.loc[(df.ttype == tumor) & (df.symbol.isin(cats[tumor][cate]))] = cate
#     df = df[df.cate_ids != ""]
    return df

In [ ]:
cats

In [ ]:
def get_diff_calls_IGV():
    df = BIG_MAF.pivot_table(index = ['patient', 'symbol', 'chr', 'pos', 'ttype', 'context65'], columns=['version'], values=['effect'], 
                   aggfunc=lambda x: ' '.join(x), fill_value=np.nan).reset_index()
    df = df[(df['effect', "M1"]  != df['effect', "M2"])]
    df.columns = ['patient', 'symbol', 'chr', 'pos', 'ttype', 'context65', 'effect_m1', 'effect_m2']
    df = df.merge(pvals.rename(columns = {'gene':'symbol', 'M1':'q_m1', 'M2':'q_m2'}), 
             how = 'left', on = ['ttype', 'symbol'])
    df['patient_sub'] = df.patient.str[:7]

    df = df.merge(mpairs_df.rename(columns = {'cohort': 'ttype', 'patient': 'patient_sub'} ), 
                how = 'left', on = ['ttype', 'patient_sub'] )
    DIFF_CALLS = df.loc[ ((df.q_m1 < 0.1) & (df.q_m2 > 0.1))| ((df.q_m2 < 0.1) & (df.q_m1 > 0.1)), 
                        ["ttype", "chr", "pos", 'symbol','context65', 'effect_m1', 'effect_m2', "q_m1", "q_m2", "tumor", "normal", "patient"]]
    ALL_DIFF = pd.DataFrame()
    # ((df.q_m1 < 0.1) & (df.q_m2 > 0.1))| ((df.q_m2 < 0.1) & (df.q_m1 > 0.1))
    def append_filters(cohort):
        df_diff = DIFF_CALLS.loc[DIFF_CALLS.ttype == cohort]
        filter_reasons = pd.read_csv("/demo-mount/m2_results/combined_res/filters/{}.tsv".format(cohort), sep = '\t').loc[:, ['chr', 'pos', 'patient', 'filters']]
        filter_reasons.patient = filter_reasons.patient.str[-23:]
        filter_reasons.chr = filter_reasons.chr.astype('int64')
    #     print(filter_reasons)
        filter_reasons.columns = ["chr", "pos", "patient", "filters"]
    #     print(filter_reasons)
    #     print(df_diff)
        df_diff = df_diff.merge(filter_reasons, how = 'left', on = ['chr', 'pos', 'patient'])
        df_diff.to_csv("/demo-mount/m2_results/combined_res/prep_IGV/{}.tsv".format(cohort), sep = '\t')
        return df_diff

    for cc in finished_cohort:
        ALL_DIFF = pd.concat([append_filters(cc), ALL_DIFF], axis=0)
    ALL_DIFF = ALL_DIFF.loc[:,['chr', 'pos', 'symbol','filters',  'tumor', 'patient', 'effect_m1', 'effect_m2', 'q_m1', 
                               'q_m2', 'ttype', 'normal', 'context65', 't_alt_count', 't_ref_count']].sort_values(["ttype", "tumor"])
    df_temp = gene_category(ALL_DIFF[~ALL_DIFF.tumor.isnull()], cats)
    df_temp['gene'] = df_temp['symbol']
    df_temp = df_temp.merge(MERGE_SIG.loc[:,['gene', 'pCL_M1', 'pCL_M2', 'ttype']], how = "left", on = ['gene', 'ttype'])
    df_temp["pCL_diff"] = -np.log10(df_temp["pCL_M2"]) + np.log10(df_temp["pCL_M1"])
    return df_temp
IGVC = get_diff_calls_IGV()

In [ ]:
def plot_category(cate, cut=20, recurrent = True):
    df = pd.DataFrame(0,columns = finished_cohort, index = BIG_SIGGENE.gene.unique())
    version = cate.split("_")[0]
    for cc in finished_cohort:
#         print(cats[cc][cate])
        for gene in cats[cc][cate]:
            if recurrent:
                if version == "M1":
                    df.loc[df.index == gene, cc] =  IGVC.loc[(IGVC.cate_ids == cate) & (~IGVC.effect_m1.isnull()) & 
                                                         (IGVC.symbol == gene) & 
                                                         (IGVC.ttype == cc) ].groupby(['chr', 'pos']).size().reset_index().rename({0:"counts"}, axis=1).counts.agg("max")
                elif version == "M2":
                    df.loc[df.index == gene, cc] =  IGVC.loc[(IGVC.cate_ids == cate) & (~IGVC.effect_m2.isnull()) & 
                                                         (IGVC.symbol == gene) & 
                                                         (IGVC.ttype == cc) ].groupby(['chr', 'pos']).size().reset_index().rename({0:"counts"}, axis=1).counts.agg("max")
            else:
#                 print(df_temp.loc[(df_temp.cate_ids == cate) & (df_temp.symbol == gene) & (df_temp.ttype == cc) , :].pCL_diff.unique())
                try:
                    df.loc[df.index == gene, cc] = df_temp.loc[(df_temp.cate_ids == cate) & (df_temp.symbol == gene) & (df_temp.ttype == cc) , :].pCL_diff.unique()
                except:
                    df.loc[df.index == gene, cc] = 0
#     return df
    df = df[df.sum(axis=1) != 0]
    df =  df.iloc[df.sum(axis=1).values.argsort()[::-1], df.sum(axis=0).values.argsort()[::-1]]
#     df = df.reset_index()
    
    print(df.shape[0])
    if df.shape[0] > cut:
        dfnew = df.iloc[1:cut, :]
    else:
        dfnew = df
    fig, ax = plt.subplots(figsize = (10, len(dfnew.index) * 0.4))
#     return df
#     print(df)
    ax = sns.heatmap(dfnew, cmap = "Blues", linewidth=2)
#     dfnew = dfnew.astype('float')
#     ax.set_yticklabels(ax.get_yticklabels(), rotation=45)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45)
    b, t = plt.ylim() # discover the values for bottom and top
    b += 0.5 # Add 0.5 to the bottom
    t -= 0.5 # Subtract 0.5 from the top
    plt.ylim(b, t) # update the ylim(bottom, top) values
    ax.set_title(cate )
    if recurrent:
        plt.savefig("{}_{}".format(cate,cut), dpi=300, bbox_inches='tight')
    return df

In [ ]:
def plot_bar(gene_type, min_count=0):
    
    df = plot_category(gene_type, 20, True)
    df = df.loc[df.index[df.sum(axis=1) > min_count].tolist(),df.columns[df.sum() > 1].tolist()]
    if df.shape[0]> 20:
        df = df.iloc[:20, :]
#     fig = plt.figure(figsize = (0.3*df.shape[0],6))
    ax = df.plot(kind='bar', stacked=True, figsize =(10,6) )
    ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    sns.despine()
    ax.set_title(gene_type)
    ax.set_xticklabels(ax.get_xticklabels(), rotation =45, ha = "right")
    plt.savefig(gene_type + "_new", dpi=150, bbox_inches='tight')
    return ax
plot_bar("M2_KCG", 1)
plot_bar("M1_KCG")
plot_bar("M2_nKCG", 1)
plot_bar("M1_nKCG")

In [ ]:
IGVC = pd.read_pickle("/home/qingzhang/files/IGVC.pkl")

In [ ]:

# ax.scatter(df['carat'], df['price'], c=df['color'].apply(lambda x: colors[x]))
def label_pvalues(cohort, df, label):
    colors = {label:'red', 'others':'grey', 'both':'orange'}
    big_size = 50
    small_size = 2
    sizes = {label:big_size,  'others':small_size, 'both':10}
    # sig genes set
    BIG_SIGGENE['q'] = BIG_SIGGENE.q.fillna(1)
    
    # labels
    interested = df.index[df[cohort] > 0].tolist()
    M1_sig_genes = BIG_SIGGENE.gene[(BIG_SIGGENE.q < 0.1) & (BIG_SIGGENE.version == "M1") & (BIG_SIGGENE.ttype == cohort)].tolist()
    M2_sig_genes = BIG_SIGGENE.gene[(BIG_SIGGENE.q < 0.1) & (BIG_SIGGENE.version == "M2") & (BIG_SIGGENE.ttype == cohort)].tolist()

    df = pd.melt(BIG_SIGGENE.loc[(BIG_SIGGENE.ttype == cohort), ["pCL", "gene", "version"]], 
                                 value_vars=['pCL'], 
                                 id_vars=['gene', 'version']).fillna(1)
    df['caller']  = df['version']
    df['sig'] = 'others'
    df['sig'][df['gene'].isin(interested)] = label
#     df['sig'][df['gene'].isin(list(set(M2_sig_genes) - set(M1_sig_genes)))] = 'M2'
    df['sig'][df['gene'].isin(list(set(M1_sig_genes) & set(M2_sig_genes)))] = 'both'
    df['cc'] = [colors[x] for x in df['sig'].tolist()]
    df['size'] = [sizes[x] for x in df['sig'].tolist()]
#     df['alpha'] = [alphas[x] for x in df['sig'].tolist()]
    df['unif'] = np.random.uniform(0, 1, df.shape[0])
    g = sns.FacetGrid(df, #col="variable", #col_order=['pFN','pCL', 'pCV'],
                      col = 'caller', col_order = ['M1', 'M2'],
#                       hue = 'sig', #hue_kws={"marker": ["s", "D"]},
#                       hue = 'sig', hue_order = ['none', 'both', 'M1-only', 'M2-only'],
                      height=4, sharey=False, sharex=False)
    g.map(qqplot,'unif', "value", "cc", "size")

#   g.add_legend()
    g.map(add_y_equals_x)
    plt.subplots_adjust(top=0.9)
    g.fig.suptitle(cohort + "  pCL")
    patch_list = [mpatches.Patch(color=colors[sig_type], label=sig_type) for sig_type in colors.keys()]
    g.add_legend(handles=patch_list, title = "colored by:", loc=7)
    g.fig.text(0.5, 0.1, "-log10(Expected p-value)", ha="center", fontsize=12)
    g.fig.text(0.1, 0.3, "-log10(Observed p-value)", rotation=90, ha="center", fontsize=12)
    plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.2)
    plt.savefig("/home/qingzhang/files/combined_pvals-{}_pCL.png".format(cohort), dpi=150)
    return(df)


In [ ]:
label_pvalues("SKCM", df_m2_nKCG, "M2_nKCG")